In [ ]:
# Bayes Classifier : 
    
    # Use Bayes Theorem. argmax (y) P(y|X) = P(X|y).p(y)/p(X)------ likelihood.prior/normalization = posterior
    # prior prob. belief without knowing anything about the data. posterior prob. is when you see the data.
    
    # hmap = not cancer.
    # Prior: P(cancer) = 0.008    Posteriror P(cancer|+) = 0.21
    # Posterior prob of cancer is significantly higher than its prior prob. The most probable hypothesis is still that the patient does not have cancer.
    
    # P(x) does not have any affects on the choice of yhead. Thefore yhead = argmax (y) P(X|y).p(y)
    
    # P(X1,X2,X3,...,Xn | y) = P(X|y)   Assume all attributes and labels are Boolean variables. We need to estimate 2*(2**n-1) parameters for likelihood. 1 also for prior.
    # You have too much things to estimate since you try to measure the joint probabilities. Can we reduce the parameters of P(x|y)?
    
    # Naive Bayes Assumption: Assume that the feature prob P(Xi|y) are independent given the class y.
    # P(X1,..,Xn) = P(X1|y).p(X2|y)....p(Xn|y)   Likelihood prob. Parameter size became 2*n. 2 for each feature. .
    # This assumption is not always correct but makes the problem much simpler to solve. In practice we can solve problem efficiently, probabilities may not be correct but classification is true.
    
    # Multiplying many probabilities --> Danger of underflow
    # Solution: Perform all computations by summing logs of prob.s rather than multiplying them.
    

# Bayes Classifier with Continuous Features : Gaussian Naive Bayes Classifier
    
    # Normal distribution. Mean & variance. Features are normally distributed.
    # MLE and Map for mean & variance.  sum xi / n    sum (xi-u)**2 / n
    .
    # If Naive Bayes does not work then you should check the correlation between features. 
    
    # You can use contour plot to understand the correlation between features. You can see how their variances change etc.
    # Denominator : sum(y)    P(Y=y)* multiplication p(Xi|Y = y)
    
    # Naive Bayes Issues:
    # Naive Bayes Missing Values : Ignore missing value. 
    # Independence Assumption: Usually features are not conditionally independent. In practive it often works well.
    # Incremental updates : Training is fast ( linear in the number of examples, features and classes ). 
    
    # Advantages : Very fast, low storage requirements. Robust to irrelevant features. Optimal if the conditional ind. assumptions old. A good dependable baseline for text classification.
    
# Naive Bayes in Text Classification: 
    
    # Naive Bayes generally used for text classification. Spam Classification.
    
    # How to represent documents ? Set of words. Words will be your features.
    # Bag of words representation : The position (order) of words in a text are ignored. Only the frequency (existence) of words are used. 
    # Bernoulli document model: a document is represented by a binary feature vector whose elements indicate absence or presence of corresponding word in the document. 
    # Multinomial document model: a document is represented by an integer feature vector whose elements indicate frequency of correspoding word in the document.
    
    # Naive Bayes makes two assumptions : Bag of Words Assumption: word position does not matter. + Conditional Independence Assumtion: Feature probs are independent given the class.
    # How many time did you see a word in that class / count of all words in that class.
    # MLE problem, zero prob. : Laplace Smoothing (add 1 smoothing) : count +1 / (total count in class + V)     V = total number of words. If a word exists in a class and does not exist in another class, you can solve it by Laplace smoothing. 
    
    
# Example : 
    # - Just plain boring
    # - Entirely predictable and lacks energy
    # - No surprises and very few laughs
    # + very powerful
    # + the most fun film of the summer
    
    # Priors : P(-) = 3/5 P(+) = 2/5 
    # tokens in - docs: 14
    # tokens in + docs : 9
    # Vocabulary size = 20
    
    # ? Predictable with no fun.
    # P(Predictable|+) = (0+1)/(9+20)
    # P(Predictable | - ) = (1+1)/(14+20)
    

In [ ]:
!pip install opendatasets
import opendatasets as od
od.download("https://www.kaggle.com/datasets/pradeeshprabhakar/preprocessed-dataset-sentiment-analysis") # download data from kaggle datasets

In [40]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords 
from nltk.stem.snowball import SnowballStemmer
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [18]:
data = pd.read_csv(r"..\EcoPreprocessed.csv")

In [19]:
data = data[["review","division"]]

In [32]:
data.loc[data.division == "neutral","division"] = "negative"

In [35]:
data

,review,division
0,abl play youtub alexa,positive
1,abl recogn indian accent realli well drop func...,positive
2,absolut smart devic amazon connect extern sub ...,positive
3,absolut amaz new member famili control home vo...,positive
4,absolut amaz previous sceptic invest money arr...,positive
...,...,...
4079,yo yo yo love go want one smart speaker valu m...,positive
4080,youtub music,negative
4081,youtub support nahi kartasong recognis achha nahi,negative
4082,yup proscontrol wipro light amazinglysoni brav...,negative


In [33]:
data.division.value_counts()

positive    3066
negative    1018
Name: division, dtype: int64

In [23]:
nltk.download("stopwords")
stop_words = stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ugury\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [24]:
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [25]:
stemmer = SnowballStemmer("english")

In [26]:
stemmer.stem("changing")

'chang'

In [27]:
def preprocess(text):
    
    text = text.lower().strip() 
    text = re.sub("\d+","",text)
    text = re.sub("[^\w\s]+","",text) 
    text = re.findall("\w+",text) 
    text = [stemmer.stem(word) for word in text if word not in stop_words]
    text = " ".join(text) 
    return text

In [28]:
sample_review = "This was a 10/10 movie! I strongly recommend. I am amazed by every scene. The cast is so strong!" 


In [29]:
print(preprocess(sample_review))

movi strong recommend amaz everi scene cast strong


In [30]:
data.review = data.review.map(preprocess)

In [31]:
data.review

0                                   abl play youtub alexa
1       abl recogn indian accent realli well drop func...
2       absolut smart devic amazon connect extern sub ...
3       absolut amaz new member famili control home vo...
4       absolut amaz previous sceptic invest money arr...
                              ...                        
4079    yo yo yo love go want one smart speaker valu m...
4080                                         youtub music
4081    youtub support nahi kartasong recognis achha nahi
4082    yup proscontrol wipro light amazinglysoni brav...
4083    zero integr capabl fire tv devic use aux usele...
Name: review, Length: 4084, dtype: object

In [36]:
data.division = data.division.replace("positive",1).replace("negative",0)

In [47]:
train_x,test_x,train_y,test_y = train_test_split(list(data.review),list(data.division),test_size = 0.2,random_state = 22)

In [48]:
train_x

['use product',
 'better go googl home cannnot makeaccept phone call alexa call place order prime connect samsung smart tv via smart thing skill basic control like onoffvolum controltechn custom care good support issuefre wipro smart light isnt realli smartonc power resum power cut need switch via appalexaon worthi use echo unlimit music amazon music',
 'everi time give error connect also play origin song rather play mashup worth googl mini far better alexa goo refund',
 'feel good good deal',
 'kid happi',
 'good product echo dot devic alexa recogn voic quick give repli connect devic also work well echo dot devic sound put also loud clear',
 'excel',
 'mani surpris alexa eco dottoo mani bug recogn voic oftentim understand command keep play exist doesnt work reset command doesnt understand request choic song pali like wish play specif album gana amazon music stick play jio savan horribl experi eco dotamazon team pls fix bug updat els custom review wont let',
 'good product amazon alexa

In [49]:
vectorizer = CountVectorizer()

In [50]:
train_x = vectorizer.fit_transform(train_x)

In [51]:
train_x

<3267x4041 sparse matrix of type '<class 'numpy.int64'>'
	with 30779 stored elements in Compressed Sparse Row format>

In [55]:
test_x = vectorizer.transform(test_x) # fit by train and transform by test.

In [56]:
model = MultinomialNB()

In [57]:
model = model.fit(train_x,train_y)

In [61]:
predictions = model.predict(test_x)

In [62]:
accuracy_score(test_y,predictions)

0.8408812729498164